**Author**: Winnie Shen <br>
**Date**: Oct 17, 2022 <br>
**Purpose**: To scrape DOB BIS job permits for total residential zoning square footage, total residential FAR, total non-residential zoning square footage, total non-residential FAR, lot area and number of parking spaces in the enclosed and open spaces categories. 

*Note*: DOB BIS's access denied error is still triggered by this scraper. Best temporary solution is to switch what wifi your computer is connected to.

In [24]:
#Set up 

from selenium import webdriver
from bs4 import BeautifulSoup as soup
import pandas as pd
import numpy as np
import re
import time
import decimal
from decimal import Decimal

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#Using this package to be less obvious with the scraping
#and to get around DOB's access denied error
#for info on the package: https://github.com/ultrafunkamsterdam/undetected-chromedriver

#pip install undetected-chromedriver

import undetected_chromedriver as uc

In [25]:
#Read CSV of job numbers 
#jobs_tbd = pd.read_csv("/Users/winnieshen/Downloads/jobs_to_be_scraped_bins.csv")
jobs_tbd = pd.read_csv("/Users/winnieshen/Downloads/tz_jobnums.csv")

#for demo
jobs_tbd = jobs_tbd.head()

In [26]:
jobs_tbd

,jobnum
0,103174084
1,103322815
2,103368160
3,103523821
4,103630215


In [27]:
#helper function

#just to make all the scraped values into uniform numbers
#also accounting for the instances where a letter is inputted rather than a number
def num_maker(string):
    string = string.replace(" ","")
    try:
        if "," in string:
            string = re.sub(",","",string)
            string = int(string)
        elif len(string) == 0:
            string = 0
        else:
            string = int(string)
        return string
    except:
        string = 0
        return string 

In [28]:
#launching the undetected version of chrome
driver = uc.Chrome(suppress_welcome = True)

In [600]:
#421384512  
#jobs_tbd['jobnum'][6041:].to_list()
#5935+106

[421384512,
 421384772,
 421393708,
 421401824,
 421405205,
 421409194,
 421423347,
 421423356,
 421426497,
 421461251,
 421513927,
 421540522,
 421551154,
 421553045,
 421678562,
 421678795,
 421679641,
 421682959,
 421692528,
 421693340,
 421717100,
 421717155]

In [29]:
#initializing the final dataframe
results = pd.DataFrame()
colnames = ["job","res_zsf","res_far","non_res_zsf","non_res_far","lot_area","enclosed_parking_check","enc_parking_spaces", "open_parking_check", "open_park_area","open_park_spaces"]
results_final = pd.DataFrame(columns = colnames)


#run this 
job = jobs_tbd['jobnum'].to_list() #this is going to be very very slow 

#Note about the scraping process: 
#this is very slow and took a few hours for almost 700 job permits
#because DOB's access denied error is still inevitably triggered
#my way around this has been switching what wifi network my computer's connected to
#no idea what that's doing but it seems to work/most reliable way to not trigger the access denied error for at least
#a hundred or so records 


for i in range(len(job)):
    
    #initializing these variables
    job_all = []
    res_zsf_all = []
    res_far_all = []
    non_res_zsf_all = []
    non_res_far_all = []
    lot_area_all = []
    res_zsf = int()
    res_far = Decimal()

    enclosed_parking_check_all = []
    enclosed_parking_all = []

    open_parking_check_all = []
    open_parking_area_all = []
    open_parking_all = []
    #variables end 
    
    #opening the bis web 
    driver.get('https://a810-bisweb.nyc.gov/bisweb/bispi00.jsp')
    
    WebDriverWait(driver, timeout = 120).until(EC.element_to_be_clickable((By.LINK_TEXT, 'DOB NOW Public Portal')))
    
    #typing in the job number 
    text_area = driver.find_element("xpath",'//*[@id="passjobnumber1"]')
    text_area.send_keys(job[i])
    
    #lots of time sleep is sprinkled throughout the scraper to help make it less obvious
    #but if that's not actually doing anything, feel free to delete
    time.sleep(2)
    
    #specifically selecting the first document related to the job number
    #the 1st document generally has the data we're interested in and later documents may not have those fields
    text_area_doc = driver.find_element("xpath",'//*[@id="passdocnumber1"]')
    text_area_doc.send_keys("01")
    
    time.sleep(2)
    
    #clicking the search button
    driver.find_element("xpath","/html/body/div/table[2]/tbody/tr[17]/td/table/tbody/tr/td[3]/input").click()
    
    #this is when we'd run into the access denied error so we're checking if those words are on the screen
    get_source = driver.page_source
    access_flag = "Access Denied"
    
    while access_flag in get_source:
        
        #this is my not always consistent work-around the access denied error
        #I found if you quit and re-open the driver and re-try the same job number
        #sometimes DOB lets you
        #if you're getting access denied like 5-6 times in a row, I'd switch wifi connections and try again 
        
        driver.quit()
        
        driver = uc.Chrome(suppress_welcome = True)
        driver.get('https://a810-bisweb.nyc.gov/bisweb/bispi00.jsp') #just need to be on bis first
        
        WebDriverWait(driver, timeout = 120).until(EC.element_to_be_clickable((By.LINK_TEXT, 'DOB NOW Public Portal')))
        
        text_area = driver.find_element("xpath",'//*[@id="passjobnumber1"]')
        text_area.send_keys(job[i])
        
        time.sleep(2)

        text_area_doc = driver.find_element("xpath",'//*[@id="passdocnumber1"]')
        text_area_doc.send_keys("01")
        
        time.sleep(2)

        driver.find_element("xpath","/html/body/div/table[2]/tbody/tr[17]/td/table/tbody/tr/td[3]/input").click()

        get_source = driver.page_source
        access_flag = "Access Denied"
        
        if access_flag not in get_source:
            WebDriverWait(driver, timeout = 120).until(EC.element_to_be_clickable((By.LINK_TEXT, 'Document Overview')))
            break
   
    else:
        WebDriverWait(driver, timeout = 120).until(EC.element_to_be_clickable((By.LINK_TEXT, 'Document Overview')))
    
    #appending job number
    job_all.append(job[i])
    

    #scraping
    #proposed use table 
    try:
        tbl = driver.find_element("xpath","/html/body/center/table[20]/tbody/tr[8]/td/table").get_attribute('outerHTML')
    except:
        tbl = driver.find_element("xpath","/html/body/center/table[20]/tbody/tr[9]/td/table").get_attribute('outerHTML')
 
    
    df  = pd.read_html(tbl)[0]

    for j in range(len(df.loc[:,1])):
        if str(df[1][j]).upper() in "RESIDENTIAL":
            #this loop adds up all the rows that have a residential use 
            res_zsf_temp = int(df[2][j])
            res_far_temp = Decimal(df[4][j])

            res_zsf = res_zsf_temp + res_zsf
            res_far = res_far_temp + res_far
           
        if str(df[1][j]) in "Proposed Totals:":
            if pd.isna(df[2][j]) is False:
                non_res_zsf = int(df[2][j])-res_zsf
            else:
                non_res_zsf = 0
            if pd.isna(df[4][j]) is False:
                non_res_far = Decimal(df[4][j])-res_far
            else:
                non_res_far = 0

            res_zsf_all.append(res_zsf)
            res_far_all.append(res_far)
            non_res_zsf_all.append(non_res_zsf)
            non_res_far_all.append(non_res_far)
        
    #lot area 
    try:
        lot_string = driver.find_element("xpath",'/html/body/center/table[20]/tbody/tr[10]/td[4]').text
    except:
        lot_string = driver.find_element("xpath",'/html/body/center/table[20]/tbody/tr[11]/td[4]').text
    
    
    lot_area = re.search('sq.ft.\):(.*)L',lot_string).group(1)

    lot_area = num_maker(lot_area)
    
    lot_area_all.append(lot_area)
    
    #enclosed parking

    #checking if there is parking 
    try:
        yes_box = driver.find_element("xpath","/html/body/center/table[20]/tbody/tr[15]/td[3]/img[1]").get_attribute("src")
    except:
        yes_box = driver.find_element("xpath","/html/body/center/table[20]/tbody/tr[16]/td[3]/img[1]").get_attribute("src") 
    
    try:
        no_box = driver.find_element("xpath","/html/body/center/table[20]/tbody/tr[15]/td[3]/img[2]").get_attribute("src")
    except:
          no_box = driver.find_element("xpath","/html/body/center/table[20]/tbody/tr[16]/td[3]/img[2]").get_attribute("src")

    if "box_check" in yes_box:
        enclosed_parking_check_all.append("Yes")
    elif "box_check" in no_box:
        enclosed_parking_check_all.append("No")
    else:
        enclosed_parking_check_all.append("No selection")
    
    #how many parking spaces 
    try:
        parking_spaces = driver.find_element("xpath",'/html/body/center/table[20]/tbody/tr[15]/td[3]').text
    except:
        parking_spaces = driver.find_element("xpath",'/html/body/center/table[20]/tbody/tr[16]/td[3]').text
    
    if re.search('spaces:(.*)',parking_spaces) is None:
        num_parking = 0
    else:
        num_parking = re.search('spaces:(.*)',parking_spaces).group(1)
        num_parking = num_maker(num_parking)
    
    enclosed_parking_all.append(num_parking)

    #open spaces parking 
    #parking area
    open_space_check = driver.find_element("xpath","/html/body/center/table[28]/tbody/tr[3]/td").text
    
    if "Not Provided" in open_space_check:
        open_parking_check_all.append("No")
        open_parking_area_all.append(0)
        open_parking_all.append(0)
    else:
        #parking spaces
        parking_area = driver.find_element("xpath","/html/body/center/table[28]/tbody/tr[5]").text
        parking_area = re.search('ft.\):(.*)P',parking_area).group(1)
        parking_area = num_maker(parking_area)

        open_parking_spaces = driver.find_element("xpath","/html/body/center/table[28]/tbody/tr[5]").text
        open_parking_spaces = re.search('no.\):(.*)',open_parking_spaces).group(1)
        open_parking_spaces = num_maker(open_parking_spaces)
        
        open_parking_check_all.append("Yes")
        open_parking_area_all.append(parking_area)
        open_parking_all.append(open_parking_spaces)
    
    #time.sleep(5)
    
    results = pd.DataFrame(list(zip(job_all,res_zsf_all,res_far_all,non_res_zsf_all,non_res_far_all, lot_area_all, enclosed_parking_check_all, enclosed_parking_all, open_parking_check_all, open_parking_area_all, open_parking_all)), columns = colnames)
    #print(results)
    
    results_final = pd.concat([results_final, results])
    
    print(str(job[i])+" is finished") #this is just for a visual check that things are working 
    
results_final
        

103174084 is finished
103322815 is finished
103368160 is finished
103523821 is finished
103630215 is finished


,job,res_zsf,res_far,non_res_zsf,non_res_far,lot_area,enclosed_parking_check,enc_parking_spaces,open_parking_check,open_park_area,open_park_spaces
0,103174084,0,0,0,0,0,No selection,0,No,0,0
0,103322815,0,0,0,0,0,No selection,0,No,0,0
0,103368160,0,0,0,0,0,No selection,0,No,0,0
0,103523821,0,0,0,0,0,No selection,0,No,0,0
0,103630215,0,0,0,0,0,No selection,0,No,0,0


In [604]:
#writing final results
#you'll probably end up combining a bunch of result dataframes unfortunately 
#results_final_large = pd.DataFrame()

#results_final.to_csv("results_1.csv")
#results_final.to_csv("results_2.csv")
#results_final.to_csv("results_3.csv")
#results_final.to_csv("results_4.csv")
# results_final.to_csv("results_5.csv")
#results_final.to_csv("results_6.csv")
#results_final.to_csv("results_7.csv")
#results_final.to_csv("results_8.csv")
#results_final.to_csv("results_9.csv")
#results_final.to_csv("results_10.csv")
#results_final.to_csv("results_11.csv")
#results_final.to_csv("results_12.csv")
#results_final.to_csv("results_13.csv")
#results_final.to_csv("results_14.csv")
#results_final.to_csv("results_15.csv")
#results_final.to_csv("results_16.csv")
#results_final.to_csv("results_17.csv")
#results_final.to_csv("results_18.csv")
#results_final.to_csv("results_19.csv")
#results_final.to_csv("results_20.csv")
#results_final.to_csv("results_21.csv")
#results_final.to_csv("results_22.csv")
#results_final.to_csv("results_23.csv")
#results_final.to_csv("results_24.csv")
#results_final.to_csv("results_25.csv")
#results_final.to_csv("results_26.csv")
#results_final.to_csv("results_27.csv")
#results_final.to_csv("results_28.csv")
#results_final.to_csv("results_29.csv")
#results_final.to_csv("results_30.csv")
#results_final.to_csv("results_31.csv")
#results_final.to_csv("results_32.csv")
#results_final.to_csv("results_33.csv")
#results_final.to_csv("results_34.csv")
#results_final.to_csv("results_35.csv")
#results_final.to_csv("results_36.csv")
#results_final.to_csv("results_37.csv")
#results_final.to_csv("results_38.csv")
#results_final.to_csv("results_39.csv")
#results_final.to_csv("results_40.csv")
#results_final.to_csv("results_41.csv")
#results_final.to_csv("results_42.csv")
#results_final.to_csv("results_43.csv")
#results_final.to_csv("results_44.csv")
#results_final.to_csv("results_45.csv")
#results_final.to_csv("results_46.csv")
results_final.to_csv("results_47.csv")

results_final_large = pd.concat([results_final_large, results_final])
results_final_large.to_csv("results_final_large.csv")


In [598]:
results_final_large #

,job,res_zsf,res_far,non_res_zsf,non_res_far,lot_area,enclosed_parking_check,enc_parking_spaces,open_parking_check,open_park_area,open_park_spaces
0,103174084,0,0,0,0,0,No selection,0,No,0,0
0,103322815,0,0,0,0,0,No selection,0,No,0,0
0,103368160,0,0,0,0,0,No selection,0,No,0,0
0,103523821,0,0,0,0,0,No selection,0,No,0,0
0,103630215,0,0,0,0,0,No selection,0,No,0,0
...,...,...,...,...,...,...,...,...,...,...,...
0,421379403,4528,1.25,0,0.00,3625,No,0,Yes,900,3
0,421380491,5852,1.23,0,0.00,5000,Yes,2,No,0,0
0,421380516,5852,1.23,0,0.00,4750,Yes,2,No,0,0
0,421382293,4995,2.00,0,0.00,2500,No,0,No,0,0
